# Step 1. Import data and Data Exploration

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print ("Shape of Training Dataset: %s" % str(mnist.train.images.shape))
print ("Shape of Training Labels: %s" % str(mnist.train.labels.shape))
print ("Shape of Test Dataset: %s" % str(mnist.test.images.shape))
print ("Shape of Test Labels: %s" % str(mnist.test.labels.shape))

Shape of Training Dataset: (55000, 784)
Shape of Training Labels: (55000, 10)
Shape of Test Dataset: (10000, 784)
Shape of Test Labels: (10000, 10)


# Step 2. Construct a tensorflow graph
It is a Fully-connected 2-Layer Neural Network [784, 10]
 * Activation: Softmax 
 * Learning Rate: 0.5
 * Loss: Cross Entropy

In [23]:
import tensorflow as tf

# Initializing tensorflow graph
graph = tf.Graph()
with graph.as_default():
    # Declaring placeholders 
    tfTrainingData = tf.placeholder(tf.float32, [None, 784])
    tfTrainingLabels = tf.placeholder(tf.float32, [None, 10])
    keep_prob = tf.placeholder(tf.float32)
    
    def weight_variable(shape):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    
    def bias_variable(shape):
        return tf.Variable(tf.constant(0.1, shape=shape))
    
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    
    x_image = tf.reshape(tfTrainingData, [-1, 28, 28, 1])
    
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    h_fc_1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    output = tf.nn.softmax(tf.matmul(h_fc_1_drop, W_fc2) + b_fc2)
    # Declaring Operations
    loss = tf.reduce_mean(-tf.reduce_sum(tfTrainingLabels * tf.log(output), reduction_indices=[1]))
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
    correctPrediction = tf.equal(tf.argmax(output, 1), tf.argmax(tfTrainingLabels, 1))
    accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))

# Step 3. Let the tensor flows!

In [22]:
# Flow the tensors here 
with tf.Session(graph=graph) as session:
    session.run(tf.initialize_all_variables())

    for step in range(2000):
        batchTrainingDataset, batchTrainingLabels = mnist.train.next_batch(50)
        _, stepLoss, stepAcc = session.run([optimizer, loss, accuracy], 
                                    feed_dict={tfTrainingData: batchTrainingDataset,
                                               tfTrainingLabels: batchTrainingLabels,
                                               keep_prob: 0.5})
        if (step % 100 == 0):
            print ("Step %5d: Loss = %10.6f, Accuracy = %10.6f" % (step, stepLoss, stepAcc))
        
    testAccuracy = session.run(accuracy, feed_dict={tfTrainingData: mnist.test.images,
                                                    tfTrainingLabels: mnist.test.labels,
                                                    keep_prob: 1.0})
    print ("Test Accuracy = %10.6f" % (testAccuracy))

Step     0: Loss =  11.110835, Accuracy =   0.140000
Step   100: Loss =   1.235339, Accuracy =   0.640000
Step   200: Loss =   0.859897, Accuracy =   0.760000
Step   300: Loss =   0.427537, Accuracy =   0.880000
Step   400: Loss =   0.330915, Accuracy =   0.900000
Step   500: Loss =   0.193836, Accuracy =   0.920000
Step   600: Loss =   0.496489, Accuracy =   0.860000
Step   700: Loss =   0.470899, Accuracy =   0.900000
Step   800: Loss =   0.265499, Accuracy =   0.920000
Step   900: Loss =   0.448943, Accuracy =   0.900000
Step  1000: Loss =   0.178400, Accuracy =   0.960000
Step  1100: Loss =   0.237944, Accuracy =   0.940000
Step  1200: Loss =   0.393328, Accuracy =   0.920000
Step  1300: Loss =   0.121720, Accuracy =   0.960000
Step  1400: Loss =   0.089794, Accuracy =   0.980000
Step  1500: Loss =   0.183843, Accuracy =   0.920000
Step  1600: Loss =   0.142925, Accuracy =   0.960000
Step  1700: Loss =   0.089637, Accuracy =   0.960000
Step  1800: Loss =   0.158342, Accuracy =   0.